In [ ]:
#uses RegisteredMultiplier  - v1.0x

In [1]:
import sys
import math
import io

In [2]:
#N>=2
N = input('Enter N: ')

Enter N: 14


In [3]:
#AdderSize>=2
AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')

Enter maximum size allowed for base adders (empty for 2): 4


In [4]:
#IN_WIDTH>=1
IN_WIDTH = input('Enter IN_WIDTH Default Value (empty for 10): ')

Enter IN_WIDTH Default Value (empty for 10): 


In [5]:
#INPUT_REG_DEPTH>=0
INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH Default Value (empty for 1): ')

Enter INPUT_REG_DEPTH Default Value (empty for 1): 


In [6]:
#MULT_PIPE_DEPTH>=0
MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH Default Value (empty for 1): ')

Enter MULT_PIPE_DEPTH Default Value (empty for 1): 


In [7]:
N = int(N)
if(N<2):
    N=2

if not AdderSize:
    AdderSize=2
else:
    AdderSize = int(AdderSize)
    if AdderSize<2:
        AdderSize=2
    if AdderSize>N:
        AdderSize=N

if not IN_WIDTH:
    IN_WIDTH=10
else:
    IN_WIDTH = int(IN_WIDTH)
    if(IN_WIDTH<1):
        IN_WIDTH=1

if not INPUT_REG_DEPTH:
    INPUT_REG_DEPTH=1
else:
    INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
    if(INPUT_REG_DEPTH<0):
        INPUT_REG_DEPTH=0

if not MULT_PIPE_DEPTH:
    MULT_PIPE_DEPTH=1
else:
    MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
    if(MULT_PIPE_DEPTH<0):
        MULT_PIPE_DEPTH=0

In [ ]:
ModuleName = "DotProduct_{}_noHR_M_A{}".format(N, AdderSize)

In [8]:
__Print_To_File = 0

if __Print_To_File<=0:
    of=sys.stdout
else:
    of=open("./"+ModuleName+".v".format(N, AdderSize), 'w+')

In [9]:
print("`timescale 1ns / 1ps\n", file=of)
print("module "+ModuleName, file=of)
print("#(", file=of)
print("parameter IN_WIDTH = {},".format(IN_WIDTH), file=of)
print("parameter INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
print("parameter MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
print(")(", file=of)
print("input clk, reset, enable,", file=of)
print("input inReady,", file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("A{}, ".format(i), end='', file=of)
print(file=of)
print("input signed [IN_WIDTH-1:0] ", end='', file=of)
for i in range(N):
    print("B{}, ".format(i), end='', file=of)
print(file=of)
print("output outReady,", file=of)
lgn=math.ceil(math.log2(N))
al=-1+lgn
if al==0:
    print("output signed [(2*IN_WIDTH):0] DP,", file=of)
elif al<0:
    print("output signed [(2*IN_WIDTH){}:0] DP,".format(al), file=of)
else:
    print("output signed [(2*IN_WIDTH)+{}:0] DP,".format(al), file=of)
print("output earlyOutReady", file=of)
print(");\n", file=of)

`timescale 1ns / 1ps

module DotProduct_14_noHR_M_A4
#(
parameter IN_WIDTH = 10,
parameter INPUT_REG_DEPTH = 1,
parameter MULT_PIPE_DEPTH = 1
)(
input clk, reset, enable,
input inReady,
input signed [IN_WIDTH-1:0] A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, 
input signed [IN_WIDTH-1:0] B0, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, B12, B13, 
output outReady,
output signed [(2*IN_WIDTH)+3:0] DP,
output earlyOutReady
);



In [10]:
AIBuf=io.StringIO()

level=0

nextElememtNo=N #remained
sizeList=[]
for i in range(N):
    sizeList.append(1)

RootList=[]
for i in range(N):
    RootList.append(i)
AdditionalRegList=[]
for i in range(N):
    AdditionalRegList.append(0)
    
i=0
new = N

while 1:
    #print("StartNextElememtNo = {}".format(nextElememtNo))
    if nextElememtNo==1:
        break
    if nextElememtNo<=AdderSize:
        lastLevel=True
    else:
        lastLevel=False        
    currentElememtNo = nextElememtNo
    nextElememtNo=0
    ORimp=False
    RegEnableImp=False
    while currentElememtNo>0:
        if currentElememtNo >= AdderSize:
            CurrentAdderSize = AdderSize
            currentElememtNo -= AdderSize
        else:
            if not lastLevel:
                CurrentAdderSize = 1
                currentElememtNo -= 1
            else:
                CurrentAdderSize = currentElememtNo
                currentElememtNo = 0
        nextElememtNo=nextElememtNo+1
        #print("nextElememtNo = {}".format(nextElememtNo))
        outSize=0
        for k in range (i, i+CurrentAdderSize):
            outSize+=sizeList[k]
        sizeList.append(outSize)
        if CurrentAdderSize==1: #just reg
            root=RootList[i]
            if root != -1:
                RootList.append(i)
                #print(root)
                while root>=N:
                    root=RootList[root]
                    #print(root)
                AdditionalRegList[root]+=1
            else:
                RootList.append(-1)
            i+=1
            RegEnableImp=True
        else: #Generate a AdderSize Adder (maybe less size for last stage)
            RootList.append(-1)
            if not lastLevel:
                if outSize==2: #outSize>=2
                    AIBuf.write("wire signed [2*IN_WIDTH:0] M{};\n".format(new))
                else:
                    AIBuf.write("wire signed [2*IN_WIDTH+{}:0] M{};\n".format(math.ceil(math.log(outSize,2))-1, new))
            inSizeL = math.ceil(math.log(sizeList[i],2))
            #print("inSize = {}".format(inSize))
            rE = i+CurrentAdderSize
            eSizeList=[]
            finalRootList=[]
            for k in range (i, rE):
                curSizeL=math.ceil(math.log(sizeList[k],2))
                eSizeL=inSizeL-curSizeL
                eSizeList.append(eSizeL)
                #print("esize = {}".format(eSizeL))
                root=RootList[k]
                if root != -1:
                    while root>=N:
                        root=RootList[root]
                else:
                    root=k
                finalRootList.append(root)
                if eSizeL!=0:
                    AIBuf.write("wire signed [2*IN_WIDTH")
                    if inSizeL!=1:
                         AIBuf.write("+{}".format(inSizeL-1, k))
                    if(root<N):
                        AIBuf.write(":0] DPpart{}e = ".format(root))
                    else:
                        AIBuf.write(":0] M{}e = ".format(root))
                    AIBuf.write("{{")
                    AIBuf.write("{}".format(eSizeL))
                    AIBuf.write("{")
                    if(root<N):
                        AIBuf.write("DPpart{}".format(root))
                    else:
                        AIBuf.write("M{}".format(root))
                    AIBuf.write("[2*IN_WIDTH".format(k))
                    if curSizeL==0:
                        AIBuf.write("-1")
                    elif curSizeL>1:
                        AIBuf.write("+{}".format(curSizeL-1))
                    AIBuf.write("]}},")
                    if(root<N):
                        AIBuf.write("DPpart{}".format(root))
                    else:
                        AIBuf.write("M{}".format(root))
                    AIBuf.write("};\n")
            AIBuf.write("Registered{}to1Adder_NIR #(.IN_WIDTH(2*IN_WIDTH".format(CurrentAdderSize))
            curSizeL=math.ceil(math.log(sizeList[i],2))
            if inSizeL>0:
                AIBuf.write("+{}".format(inSizeL))
            AIBuf.write("))\nAdd{}(\nclk, reset, enable,\n".format(new))
            if level==0:
                AIBuf.write("MOR,\n")
            else:
                AIBuf.write("ORAdd[{}],\n".format(level-1))
            for k in range (i, rE):
                if(finalRootList[k-i]<N):
                    AIBuf.write("DPpart{}".format(finalRootList[k-i]))
                else:
                    AIBuf.write("M{}".format(finalRootList[k-i]))
                if(eSizeList[k-i]!=0):
                    AIBuf.write("e");
                AIBuf.write(", ")
            AIBuf.write("\n")
            if lastLevel:
                AIBuf.write("outReady,\n")
            else:
                if ORimp==False:
                    AIBuf.write("ORAdd[{}],\n".format(level))
                    ORimp=True
                else:
                    AIBuf.write("ORa{}, //not used\n".format(new))
            if lastLevel:
                AIBuf.write("DP,\n".format(new))
            else:
                AIBuf.write("M{},\n".format(new))
            if lastLevel:
                AIBuf.write("earlyOutReady);\n\n")        
            else :
                AIBuf.write("aeor{}); //not used\n\n".format(new))
            i+=CurrentAdderSize
        new+=1
    level+=1

#print(RootList)
#print(AdditionalRegList)

In [11]:
print("wire signed [(2*IN_WIDTH)-1:0] ", end='', file=of)
for i in range (N):    
    print("DPpart{}".format(i), end='', file=of)
    if i==N-1:
        print(";\n", file=of)              
    else:
        print(", ", end='', file=of)
print("wire MOR;", file=of)
print(file=of)
for i in range (N):
    print("RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),", file=of)
    print(".INPUT_REG_DEPTH(INPUT_REG_DEPTH", end='', file=of)
    IARD = AdditionalRegList[i];
    if IARD!=0:
         print("+{}".format(IARD), end='', file=of)
    print("),\n.MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))", file=of)
    print("M{}(clk, reset, enable,".format(i), file=of)
    print("inReady,", file=of)
    print("A{}, B{},".format(i, i), file=of)
    if i==0:
        print("MOR,", file=of)
    else:
        print("MOR{}, //not used".format(i), file=of)
    print("DPpart{},".format(i), file=of)
    print("eMOR{}); //not used".format(i), file=of)
    print(file=of)

wire signed [(2*IN_WIDTH)-1:0] DPpart0, DPpart1, DPpart2, DPpart3, DPpart4, DPpart5, DPpart6, DPpart7, DPpart8, DPpart9, DPpart10, DPpart11, DPpart12, DPpart13;

wire MOR;

RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),
.INPUT_REG_DEPTH(INPUT_REG_DEPTH),
.MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))
M0(clk, reset, enable,
inReady,
A0, B0
MOR,
DPpart0,
eMOR0 //not used);

RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),
.INPUT_REG_DEPTH(INPUT_REG_DEPTH),
.MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))
M1(clk, reset, enable,
inReady,
A1, B1
MOR1, //not used
DPpart1,
eMOR1 //not used);

RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),
.INPUT_REG_DEPTH(INPUT_REG_DEPTH),
.MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))
M2(clk, reset, enable,
inReady,
A2, B2
MOR2, //not used
DPpart2,
eMOR2 //not used);

RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),
.INPUT_REG_DEPTH(INPUT_REG_DEPTH),
.MULT_PIPE_DEPTH(MULT_PIPE_DEPTH))
M3(clk, reset, enable,
inReady,
A3, B3
MOR3, //not used
DPpart3,
eMOR3 //not used);

RegisteredMultiplier #(.IN_WIDTH(IN_WIDTH),

In [12]:
if level>1:
    print("wire [0:{}] ORAdd;\n".format(level-2), file=of)
print(AIBuf.getvalue(), end='',file=of)

wire [0:1] ORAdd;

wire signed [2*IN_WIDTH+1:0] M14;
Registered4to1Adder_NIR #(.IN_WIDTH(2*IN_WIDTH))
Add14(
clk, reset, enable,
MOR,
DPpart0, DPpart1, DPpart2, DPpart3, 
ORAdd[0],
M14,
aeor14); //not used

wire signed [2*IN_WIDTH+1:0] M15;
Registered4to1Adder_NIR #(.IN_WIDTH(2*IN_WIDTH))
Add15(
clk, reset, enable,
MOR,
DPpart4, DPpart5, DPpart6, DPpart7, 
ORa15, //not used
M15,
aeor15); //not used

wire signed [2*IN_WIDTH+1:0] M16;
Registered4to1Adder_NIR #(.IN_WIDTH(2*IN_WIDTH))
Add16(
clk, reset, enable,
MOR,
DPpart8, DPpart9, DPpart10, DPpart11, 
ORa16, //not used
M16,
aeor16); //not used

wire signed [2*IN_WIDTH+3:0] M19;
wire signed [2*IN_WIDTH+1:0] DPpart12e = {{2{DPpart12[2*IN_WIDTH-1]}},DPpart12};
Registered4to1Adder_NIR #(.IN_WIDTH(2*IN_WIDTH+2))
Add19(
clk, reset, enable,
ORAdd[0],
M14, M15, M16, DPpart12e, 
ORAdd[1],
M19,
aeor19); //not used

wire signed [2*IN_WIDTH+3:0] DPpart13e = {{4{DPpart13[2*IN_WIDTH-1]}},DPpart13};
Registered2to1Adder_NIR #(.IN_WIDTH(2*IN_WIDTH+4))
A

In [13]:
print("endmodule", file=of)

endmodule


In [14]:
if __Print_To_File>0:
    of.close()